In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday
from pandas.tseries.offsets import CustomBusinessDay
import datetime

# Caminhos dos arquivos
arquivo_principal = "Cópia de REL_ATESTES_CYRELA_20241114 - Copia.xlsx"
arquivo_referencia = "Copia_Ateste_Robo.xlsx"

# Configuração do calendário para considerar feriados nacionais e de São Paulo
class FeriadosSP(AbstractHolidayCalendar):
    rules = [
        Holiday("Ano Novo", month=1, day=1),
        Holiday("Aniversário de São Paulo", month=1, day=25),
        Holiday("Tiradentes", month=4, day=21),
        Holiday("Dia do Trabalho", month=5, day=1),
        Holiday("Independência do Brasil", month=9, day=7),
        Holiday("Nossa Senhora Aparecida", month=10, day=12),
        Holiday("Finados", month=11, day=2),
        Holiday("Proclamação da República", month=11, day=15),
        Holiday("Natal", month=12, day=25),
    ]

# Criar calendário customizado
feriados = FeriadosSP().holidays(start="2024-01-01", end="2024-12-31")
dias_uteis = CustomBusinessDay(holidays=feriados)

# Funções auxiliares
def tratar_apf(apf):
    """Trata e normaliza o número da APF"""
    return str(apf).replace(".", "").replace("-", "").lstrip("0")

def encontrar_data_proxima(dados, coluna_data):
    """Encontra a data mais próxima para o mesmo APF"""
    dados_validos = dados[dados[coluna_data].notna()]
    if not dados_validos.empty:
        return dados_validos[coluna_data].min()
    return None

def calcular_data_util(data_vencimento):
    """Calcula a data 4 dias úteis antes da data de vencimento"""
    if pd.isna(data_vencimento):
        return None
    return (data_vencimento - 4 * dias_uteis).date()

try:
    # Carregar as planilhas com pandas
    df_principal = pd.read_excel(arquivo_principal)
    df_referencia = pd.read_excel(arquivo_referencia, sheet_name="Acompanhamento Ateste")

    # Variável global para armazenar o vencimento_ateste antigo
    vencimento_ateste_antigo = None

    # Converter as colunas de datas para o formato datetime no pandas
    df_principal["VALIDADE DO ATESTE"] = pd.to_datetime(df_principal["VALIDADE DO ATESTE"], errors="coerce")
    df_principal["JUNTA COMERCIAL"] = pd.to_datetime(df_principal["JUNTA COMERCIAL"], errors="coerce")

    # Validar as colunas esperadas
    colunas_esperadas_principal = [
        "APF", "VALIDADE DO ATESTE", "CND IPTU – CERTIDÃO NEGATIVA DE TRIBUTOS IMÓVEL",
        "MATRÍCULA DO IMÓVEL", "JUNTA COMERCIAL"
    ]
    colunas_esperadas_referencia = [
        "APF", "VENCIMENTO ATESTE", "CND IPTU", "MATRICULA", "JUNTA COMERCIAL"
    ]

    for coluna in colunas_esperadas_principal:
        if coluna not in df_principal.columns:
            raise KeyError(f"A coluna '{coluna}' não foi encontrada na planilha principal.")
    
    for coluna in colunas_esperadas_referencia:
        if coluna not in df_referencia.columns:
            raise KeyError(f"A coluna '{coluna}' não foi encontrada na planilha de referência.")

    # Carregar a planilha Copia_Ateste_Robo.xlsx usando openpyxl para edição
    wb = load_workbook(arquivo_referencia)
    ws = wb["Acompanhamento Ateste"]
 

    # Definir o título em negrito na coluna 20
    ws.cell(row=1, column=20).font = Font(bold=True)
    
    # Adicionar o título "Ateste Atualizado" na nova coluna (coluna 20)
    ws.cell(row=1, column=20).value = "Ateste Atualizado"

    # Estilos para "Sim" e "Não"
    verde_negrito = Font(bold=True, color="00FF00")  # Verde
    vermelho_negrito = Font(bold=True, color="FF0000")  # Vermelho

    # Atualizar as colunas na planilha de referência
    for row_num, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column), 2):
        apf_referencia = row[2].value  # APF está na 3ª coluna (index 2)
        if not apf_referencia:
            continue

        apf_tratado = tratar_apf(apf_referencia)
            # Salvar o valor antigo de "VENCIMENTO ATESTE" na variável global
        vencimento_ateste_antigo = row[11].value  # "VENCIMENTO ATESTE" na 12ª coluna
        print(vencimento_ateste_antigo)

        # Buscar dados na planilha principal
        dados_principal = df_principal[df_principal['APF'].apply(tratar_apf) == apf_tratado]
        if not dados_principal.empty:
            validade_ateste = dados_principal.iloc[0]["VALIDADE DO ATESTE"]
            junta_comercial = dados_principal.iloc[0]["JUNTA COMERCIAL"]
            if pd.isna(junta_comercial):
                junta_comercial = encontrar_data_proxima(dados_principal, "JUNTA COMERCIAL")
            cnd_iptu = dados_principal.iloc[0]["CND IPTU – CERTIDÃO NEGATIVA DE TRIBUTOS IMÓVEL"]
            matricula = dados_principal.iloc[0]["MATRÍCULA DO IMÓVEL"]

            # Atualizar as colunas na planilha
            if pd.notna(validade_ateste):
                ws.cell(row=row_num, column=12).value = validade_ateste
                ws.cell(row=row_num, column=12).number_format = "DD/MM/YYYY"
            if pd.notna(junta_comercial):
                ws.cell(row=row_num, column=8).value = junta_comercial
                ws.cell(row=row_num, column=8).number_format = "DD/MM/YYYY"
            ws.cell(row=row_num, column=9).value = cnd_iptu
            ws.cell(row=row_num, column=7).value = matricula

            # Comparar a data da coluna "VENCIMENTO ATESTE" com "VALIDADE DO ATESTE" e atualizar "Ateste Atualizado"
            if pd.notna(validade_ateste) and pd.notna(vencimento_ateste_antigo):
                # Comparar as datas sem considerar a hora
                if validade_ateste.date() == vencimento_ateste_antigo.date():
                    ws.cell(row=row_num, column=20).value = "Não"
                    ws.cell(row=row_num, column=20).font = vermelho_negrito  # "Não" em vermelho
                else:
                    ws.cell(row=row_num, column=20).value = "Sim"
                    ws.cell(row=row_num, column=20).font = verde_negrito  # "Sim" em verde

            # Atualizar a coluna "VENCIMENTO ATESTE" com nova data, se necessário
            nova_data_vencimento = calcular_data_util(vencimento_ateste_antigo)
            if nova_data_vencimento:
                ws.cell(row=row_num, column=10).value = f"ABRIR {nova_data_vencimento.strftime('%d/%m')}"
                ws.cell(row=row_num, column=10).number_format = "DD/MM/YYYY"

    # Salvar a planilha atualizada
    wb.save(arquivo_referencia)
    print("Atualização concluída com sucesso!")

except FileNotFoundError as e:
    print(f"Erro: {e}")
except KeyError as e:
    print(f"Erro de coluna ausente: {e}")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")


2024-11-10 00:00:00
2024-11-21 00:00:00
2024-11-22 00:00:00
2024-11-22 00:00:00
Atualização concluída com sucesso!


C:\Users\robson.miranda\AppData\Local\Temp\ipykernel_17720\2994548147.py:57: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_principal["VALIDADE DO ATESTE"] = pd.to_datetime(df_principal["VALIDADE DO ATESTE"], errors="coerce")
